In [17]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

DIRECTORY = r"/content/drive/My Drive/Colab Notebooks/Face-Mask-Detection/dataset"
CATEGORIES = ["with_mask", "without_mask"]

In [22]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
data = list()
labels = list()
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for i,img in enumerate(os.listdir(path)): 
        if not img.endswith('.jpg') : continue
        print('{0} ==> {1}'.format(category,i))
        image = load_img(os.path.join(path, img), target_size=(224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)
        data.append(image)
        labels.append(category) 

[INFO] loading images...
with_mask ==> 0
with_mask ==> 1
with_mask ==> 2
with_mask ==> 3
with_mask ==> 4
with_mask ==> 6
with_mask ==> 7
with_mask ==> 8
with_mask ==> 9
with_mask ==> 10
with_mask ==> 11
with_mask ==> 12
with_mask ==> 13
with_mask ==> 14
with_mask ==> 16
with_mask ==> 17
with_mask ==> 18
with_mask ==> 19
with_mask ==> 20
with_mask ==> 21
with_mask ==> 22
with_mask ==> 23
with_mask ==> 24
with_mask ==> 25
with_mask ==> 27
with_mask ==> 28
with_mask ==> 29
with_mask ==> 30
with_mask ==> 31
with_mask ==> 32
with_mask ==> 33
with_mask ==> 34
with_mask ==> 35
with_mask ==> 36
with_mask ==> 37
with_mask ==> 38
with_mask ==> 39
with_mask ==> 40
with_mask ==> 42
with_mask ==> 43
with_mask ==> 44
with_mask ==> 45
with_mask ==> 46
with_mask ==> 47
with_mask ==> 48
with_mask ==> 50
with_mask ==> 51
with_mask ==> 52
with_mask ==> 53
with_mask ==> 54
with_mask ==> 55
with_mask ==> 56
with_mask ==> 57
with_mask ==> 58
with_mask ==> 59
with_mask ==> 60
with_mask ==> 61
with_mask ==> 6

In [23]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data, dtype="float32")
labels = np.array(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)

In [24]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [25]:
# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
input_tensor=Input(shape=(224, 224, 3)))

9412608/9406464 [==============================] - 0s 0us/step


In [26]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

In [27]:
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

In [28]:
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

In [ ]:
# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the head of the network
print("[INFO] training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

[INFO] compiling model...
[INFO] training head...
Epoch 1/20
92/92 [==============================] - 140s 2s/step - loss: 0.3992 - accuracy: 0.8173 - val_loss: 0.1174 - val_accuracy: 0.9744
Epoch 2/20
92/92 [==============================] - 138s 1s/step - loss: 0.1253 - accuracy: 0.9560 - val_loss: 0.0680 - val_accuracy: 0.9744
Epoch 3/20
92/92 [==============================] - 138s 1s/step - loss: 0.0950 - accuracy: 0.9655 - val_loss: 0.0552 - val_accuracy: 0.9825
Epoch 4/20
92/92 [==============================] - 137s 1s/step - loss: 0.0662 - accuracy: 0.9826 - val_loss: 0.0504 - val_accuracy: 0.9838
Epoch 5/20
92/92 [==============================] - 139s 2s/step - loss: 0.0670 - accuracy: 0.9795 - val_loss: 0.0462 - val_accuracy: 0.9825
Epoch 6/20
92/92 [==============================] - 138s 1s/step - loss: 0.0481 - accuracy: 0.9833 - val_loss: 0.0429 - val_accuracy: 0.9811
Epoch 7/20
92/92 [==============================] - 139s 2s/step - loss: 0.0487 - accuracy: 0.9850 - val

In [ ]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs, target_names=['with_mask']))

# serialize the model to disk
print("[INFO] saving mask detector model...")
model.save("mask_detector.model", save_format="h5")

In [ ]:
lb.classes_

In [ ]:
# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")